In [1]:
import pandas as pd

In [2]:
from os import listdir

# get list of all files in the scripts directory
files = [f for f in listdir('\\scripts\\')]

In [3]:
len(files)

2572

In [152]:
# create a dictionary with key as filename and script as value
scripts_dict = {}
for fl in files:
    file_path = 'scripts\\' + fl
    with open(file_path) as f:
        txt = f.read()
        scripts_dict[fl] = txt


In [156]:
# convert the dictionary to a pandas dataframe
scripts_df = pd.DataFrame(scripts_dict.items(), columns=['genre_movie_name', 'script'])

In [157]:
scripts_df.head()

,genre_movie_name,script
0,action_collateral-damage.txt,COLLATERAL DAMAGE\n ...
1,comedy_very-bad-things.txt,"""VERY BAD ..."
2,comedy_lost-in-translation.txt,\n\n\n\n\n\n\n ...
3,mystery_citizen-kane.txt,\n\n Citizen ...
4,drama_pearl-harbor.txt,\n\n\n\n by\n\n ...


In [162]:
# remove .txt suffix from movie_name column
scripts_df['movie_name'] = scripts_df['genre_movie_name'].apply(lambda x: x.replace('.txt', '').split('_')[1])
scripts_df['genre'] = scripts_df['genre_movie_name'].apply(lambda x: x.replace('.txt', '').split('_')[0])

In [163]:
scripts_df = scripts_df.drop('genre_movie_name', axis=1)

In [167]:
genre_combined = pd.DataFrame(scripts_df[['movie_name', 'genre']].groupby(['movie_name']).agg(list)).reset_index()
scripts_df = scripts_df.drop('genre', axis=1)

scripts_df = scripts_df.sort_values(by=['movie_name'])
scripts_df = scripts_df.drop_duplicates(subset=['movie_name'], keep='last')
scripts_df.shape

(957, 2)

In [168]:
scripts_df = pd.merge(scripts_df, genre_combined, on='movie_name', how='left')

In [169]:
scripts_df.head()

,script,movie_name,genre
0,written by Karen McCullah Lutz...,10-things-i-hate-about-you,"[comedy, romance]"
1,\n\nFin of red 1957 Chevy Impala convertible d...,12,[comedy]
2,\n\n ...,12-and-holding,[drama]
3,\n\n\t\t An original screenplay by\n\...,12-monkeys,"[drama, thriller, science-fiction]"
4,\n \n\n\n\n\n ...,127-hours,"[drama, adventure, thriller]"


In [170]:
scripts_df.shape

(957, 3)

In [160]:
names_df = pd.DataFrame(scripts_df.copy()['movie_name'])

In [161]:
names_df

,movie_name
0,collateral-damage
1,very-bad-things
2,lost-in-translation
3,citizen-kane
4,pearl-harbor
...,...
2567,anastasia
2568,the-ladykillers
2569,the-talented-mr-ripley
2570,watchmen


In [32]:
box_df = pd.read_csv('boxoffice_data_clean.csv')

In [33]:
import re
def preprocess_title(x):
    if isinstance(x, str):
        x = x.lower()
        x = x.replace("'", '')
        x = re.sub('\W+\s*', ' ', x)
        x = x.replace(' ', '-')
        return x
    else:
        return ''

In [34]:
box_df['movie_name'] = box_df['title'].apply(preprocess_title)

In [35]:
box_df

,Unnamed: 0,title,lifetime_gross,year,movie_name
0,0,Star Wars: The Force Awakens,936662225,2015,star-wars-the-force-awakens
1,1,Avatar,760507625,2009,avatar
2,2,Black Panther,700059566,2018,black-panther
3,3,Avengers: Infinity War,678681680,2018,avengers-infinity-war
4,4,Titanic,659363944,1997,titanic
...,...,...,...,...,...
15738,15738,Dog Eat Dog,80,2009,dog-eat-dog
15739,15739,Paranoid Girls,78,2015,paranoid-girls
15740,15740,Confession of a Child of the Century,74,2015,confession-of-a-child-of-the-century
15741,15741,Storage 24,72,2013,storage-24


In [62]:
partial_box = pd.read_csv('matched_titles - box.csv')
partial_box

,movie_title,movie_name,match_score
0,evil-dead-ii-dead-by-dawn,evil-dead-2,56
1,friday-the-13th-part-viii-jason-takes-manhattan,friday-the-13th-part-viii,69
2,rambo-first-blood-ii-the-mission,rambo-first-blood-part-ii,70
3,the-adventures-of-buckaroo-banzai-across-the-e...,the-adventures-of-buckaroo-banzai,70
4,scarface,scarface-1983-,73
...,...,...,...
109,alien-3,alien-3,100
110,kate-and-leopold,kate-and-leopold,100
111,187,187,100
112,simone,simone,100


In [113]:
names_df = names_df.drop_duplicates()

In [114]:
names_df

,movie_name
0,collateral-damage
1,very-bad-things
2,lost-in-translation
3,citizen-kane
4,pearl-harbor
...,...
2478,the-elephant-man
2488,frequency
2513,amadeus
2535,thirteen-days


In [127]:
tmp_df = pd.merge(box_df, partial_box, on='movie_name', how='left')
tmp_df['movie_title'].fillna('0', inplace=True)
tmp_df['movie_name_new'] = tmp_df.apply(lambda x: x['movie_name'] if x['movie_title']=='0' else x['movie_title'], axis=1)
tmp_df['movie_name'] = tmp_df['movie_name_new']

joint = pd.merge(names_df, tmp_df, on='movie_name', how='left')
joint[~joint['title'].isnull()]['movie_name'].nunique()
joint = joint.drop_duplicates()

In [128]:
print(joint.shape)

(961, 8)


In [48]:
joint1 = pd.merge(names_df, box_df, on='movie_name', how='left')

In [49]:
joint1

,movie_name,Unnamed: 0,title,lifetime_gross,year
0,collateral-damage,2052.0,Collateral Damage,40077257.0,2002.0
1,very-bad-things,4669.0,Very Bad Things,9898412.0,1998.0
2,lost-in-translation,1850.0,Lost in Translation,44585453.0,2003.0
3,citizen-kane,7055.0,Citizen Kane,1585634.0,1941.0
4,pearl-harbor,198.0,Pearl Harbor,198542554.0,2001.0
...,...,...,...,...,...
2579,anastasia,1383.0,Anastasia,58406347.0,1997.0
2580,the-ladykillers,2061.0,The Ladykillers,39799191.0,2004.0
2581,the-talented-mr-ripley,912.0,The Talented Mr. Ripley,81298265.0,1999.0
2582,watchmen,615.0,Watchmen,107509799.0,2009.0


In [50]:
unmatched = joint1[joint1['title'].isnull()]['movie_name']

In [51]:
unmatched

6                              john-q
7                    perfect-creature
9                           evil-dead
22                 cowboys-and-aliens
35                love-and-basketball
                    ...              
2560                 orgy-of-the-dead
2569                      heavy-metal
2572    star-wars-revenge-of-the-sith
2576               grosse-point-blank
2577           the-planet-of-the-apes
Name: movie_name, Length: 494, dtype: object

In [116]:
mv_df = pd.read_csv('movies_metadata_clean.csv')
mv_df['title'].fillna('')
mv_df['movie_name'] = mv_df['title'].apply(preprocess_title)

In [117]:
mv_df

,Unnamed: 0,belongs_to_collection,genres,original_title,production_companies,release_date,revenue,runtime,spoken_languages,title,vote_average,vote_count,movie_name
0,0,1,"['Comedy', 'Family', 'Animation']",Toy Story,['Pixar Animation Studios'],1995-10-30,373554033.0,81.0,['English'],Toy Story,7.7,5415.0,toy-story
1,1,0,"['Fantasy', 'Family', 'Adventure']",Jumanji,"['Interscope Communications', 'TriStar Picture...",1995-12-15,262797249.0,104.0,"['Français', 'English']",Jumanji,6.9,2413.0,jumanji
2,2,1,"['Comedy', 'Romance']",Grumpier Old Men,"['Lancaster Gate', 'Warner Bros.']",1995-12-22,0.0,101.0,['English'],Grumpier Old Men,6.5,92.0,grumpier-old-men
3,3,0,"['Drama', 'Comedy', 'Romance']",Waiting to Exhale,['Twentieth Century Fox Film Corporation'],1995-12-22,81452156.0,127.0,['English'],Waiting to Exhale,6.1,34.0,waiting-to-exhale
4,4,1,['Comedy'],Father of the Bride Part II,"['Touchstone Pictures', 'Sandollar Productions']",1995-02-10,76578911.0,106.0,['English'],Father of the Bride Part II,5.7,173.0,father-of-the-bride-part-ii
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,45461,0,"['Drama', 'Family']",رگ خواب,[],NaN,0.0,90.0,['فارسی'],Subdue,4.0,1.0,subdue
45462,45462,0,['Drama'],Siglo ng Pagluluwal,['Sine Olivia'],2011-11-17,0.0,360.0,[''],Century of Birthing,9.0,3.0,century-of-birthing
45463,45463,0,"['Drama', 'Action', 'Thriller']",Betrayal,['American World Pictures'],2003-08-01,0.0,90.0,['English'],Betrayal,3.8,6.0,betrayal
45464,45464,0,[],Satana likuyushchiy,['Yermoliev'],1917-10-21,0.0,87.0,[],Satan Triumphant,0.0,0.0,satan-triumphant


In [138]:
partial_mv = pd.read_csv('matched_titles - mv.csv')
partial_mv
tmp_df = pd.merge(mv_df, partial_mv, on='movie_name', how='left')
tmp_df['movie_title'].fillna('0', inplace=True)
tmp_df['movie_name_new'] = tmp_df.apply(lambda x: x['movie_name'] if x['movie_title']=='0' else x['movie_title'], axis=1)
tmp_df['movie_name'] = tmp_df['movie_name_new']

joint_ = pd.merge(joint, tmp_df, on='movie_name', how='left')
joint_
joint_[~joint_['vote_average'].isnull()]['movie_name'].nunique()


925

In [131]:
joint_.drop_duplicates().shape

(1170, 23)

In [54]:
joint2 = pd.merge(names_df, mv_df, on='movie_name', how='left')

In [55]:
unmatched2 = joint2[joint2['title'].isnull()]['movie_name']

In [56]:
unmatched2

7                        perfect-creature
23                     cowboys-and-aliens
39                    love-and-basketball
48                      godfather-part-ii
57               pirates-of-the-caribbean
                      ...                
3110                    twilight-new-moon
3115    star-wars-the-empire-strikes-back
3141        star-wars-revenge-of-the-sith
3145                   grosse-point-blank
3146               the-planet-of-the-apes
Name: movie_name, Length: 308, dtype: object

In [57]:
unmatched_all = list(set(list(unmatched) + list(unmatched2)))

In [58]:
len(unmatched_all)

199

In [59]:
unmatched_df = pd.DataFrame(unmatched_all)

In [60]:
unmatched_df.columns = ['movie_title']

In [ ]:
! pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz

def fuzzy_match(str1, str2):
    return fuzz.ratio(str1, str2)

In [ ]:
unmatched_df['tmp'] = 1
box_df['tmp'] = 1
mv_df['tmp'] = 1

In [ ]:
unmatched_box = pd.merge(unmatched_df, box_df, how='outer', on='tmp')

In [ ]:
unmatched_box.shape

In [ ]:
unmatched_box

In [ ]:
unmatched_box['match_score'] = unmatched_box.apply(lambda x: fuzzy_match(x['movie_title'], x['movie_name']), axis=1)

In [ ]:
unmatched_box[unmatched_box['match_score'] >= 70]

In [ ]:
unmatched_mv = pd.merge(unmatched_df, mv_df, how='outer', on='tmp')

In [ ]:
unmatched_mv.shape

In [ ]:
unmatched_mv['match_score'] = unmatched_mv.apply(lambda x: fuzzy_match(x['movie_title'], x['movie_name']), axis=1)

In [ ]:
unmatched_mv[unmatched_mv['match_score'] >= 60]

In [ ]:
# unmatched_mv.to_csv('unmatched_mv.csv')

In [ ]:
# unmatched_box.to_csv('unmatched_box.csv')

In [139]:
joint_.columns

Index(['movie_name', 'Unnamed: 0_x', 'title_x', 'lifetime_gross', 'year',
       'movie_title_x', 'match_score_x', 'movie_name_new_x', 'Unnamed: 0_y',
       'belongs_to_collection', 'genres', 'original_title',
       'production_companies', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'title_y', 'vote_average', 'vote_count',
       'movie_title_y', 'match_score_y', 'movie_name_new_y'],
      dtype='object')

In [140]:
joint_ = joint_.drop(['Unnamed: 0_x', 'title_x',
                      'movie_title_x', 'match_score_x', 
                      'movie_name_new_x', 'Unnamed: 0_y', 
                      'title_y', 'movie_title_y', 
                      'match_score_y', 'movie_name_new_y'], 
                     axis=1)

In [141]:
joint_.head()

,movie_name,lifetime_gross,year,belongs_to_collection,genres,original_title,production_companies,release_date,revenue,runtime,spoken_languages,vote_average,vote_count
0,collateral-damage,40077257.0,2002.0,0.0,"['Drama', 'Action', 'Thriller']",Collateral Damage,"['David Foster Productions', 'Hacienda Product...",2002-02-06,78382433.0,108.0,"['Español', 'English']",5.5,431.0
1,very-bad-things,9898412.0,1998.0,0.0,"['Comedy', 'Crime', 'Thriller']",Very Bad Things,"['Ballpark Productions Partnership', 'Intersco...",1998-09-11,9898412.0,100.0,['English'],6.2,182.0
2,lost-in-translation,44585453.0,2003.0,0.0,['Drama'],Lost in Translation,['American Zoetrope'],2003-08-31,119723856.0,102.0,"['日本語', 'English']",7.3,1943.0
3,citizen-kane,1585634.0,1941.0,0.0,"['Mystery', 'Drama']",Citizen Kane,"['Mercury Productions', 'RKO Radio Pictures']",1941-04-30,23217674.0,119.0,['English'],8.0,1244.0
4,pearl-harbor,198542554.0,2001.0,0.0,"['War', 'Romance', 'History']",Pearl Harbor,"['Touchstone Pictures', 'Jerry Bruckheimer Fil...",2001-05-21,449220945.0,183.0,"['Français', '日本語', 'English']",6.6,1833.0


In [142]:
joint_ = joint_.drop_duplicates()
joint_.shape

(1169, 13)

In [146]:
joint_ = joint_.sort_values(by=['movie_name','year'])
joint_ = joint_.drop_duplicates(subset=['movie_name'], keep='last')

In [171]:
joint_.shape

(957, 13)

In [186]:
merged_df = pd.merge(scripts_df, joint_, on='movie_name', how='left')

In [187]:
merged_df.head()

,script,movie_name,genre,lifetime_gross,year,belongs_to_collection,genres,original_title,production_companies,release_date,revenue,runtime,spoken_languages,vote_average,vote_count
0,written by Karen McCullah Lutz...,10-things-i-hate-about-you,"[comedy, romance]",38178166.0,1999.0,0.0,"['Drama', 'Comedy', 'Romance']",10 Things I Hate About You,"['Jaret Entertainment', 'Mad Chance', 'Touchst...",1999-03-30,53478166.0,97.0,"['Français', 'English']",7.3,1768.0
1,\n\nFin of red 1957 Chevy Impala convertible d...,12,[comedy],125120.0,2009.0,0.0,"['Drama', 'Crime']",12,"['venezia 64', 'Mosfilm']",2007-06-06,7537453.0,159.0,"['', 'Pусский', 'Český']",6.6,54.0
2,\n\n ...,12-and-holding,[drama],96464.0,2006.0,0.0,['Drama'],Twelve and Holding,['Serenade Films'],2005-09-11,95687.0,90.0,['English'],7.0,20.0
3,\n\n\t\t An original screenplay by\n\...,12-monkeys,"[drama, thriller, science-fiction]",57141459.0,1995.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,\n \n\n\n\n\n ...,127-hours,"[drama, adventure, thriller]",18335230.0,2010.0,0.0,"['Drama', 'Adventure', 'Thriller']",127 Hours,"['Fox Searchlight Pictures', 'Cloud Eight Film...",2010-11-05,35692920.0,94.0,['English'],7.0,2668.0


In [188]:
merged_df.shape

(957, 15)

In [189]:
merged_df.to_csv('merged_box_mv_script.csv')

In [182]:
rt_df = pd.read_csv('rotten_tomatoes - Sheet2 (1).csv')
rt_df['movie_name'] = rt_df['movie_name'].apply(lambda x: x.replace('_', '-'))

In [183]:
rt_df

,movie_name,rt_critic_score,rt_audience_score
0,the-losers,48.0,54.0
1,the-blast-from-the-past,58.0,56.0
2,x-men-origins-wolverine,38.0,58.0
3,hellraiser-3-hell-on-earth,42.0,35.0
4,mulholland-drive,84.0,87.0
...,...,...,...
948,warrior,84.0,92.0
949,the-siege,44.0,53.0
950,soldier,13.0,44.0
951,out-of-sight,94.0,74.0


In [190]:
merged_df = pd.merge(merged_df, rt_df, on='movie_name', how='left')

In [192]:
merged_df[~merged_df['rt_critic_score'].isnull()]

,script,movie_name,genre,lifetime_gross,year,belongs_to_collection,genres,original_title,production_companies,release_date,revenue,runtime,spoken_languages,vote_average,vote_count,rt_critic_score,rt_audience_score
0,written by Karen McCullah Lutz...,10-things-i-hate-about-you,"[comedy, romance]",38178166.0,1999.0,0.0,"['Drama', 'Comedy', 'Romance']",10 Things I Hate About You,"['Jaret Entertainment', 'Mad Chance', 'Touchst...",1999-03-30,53478166.0,97.0,"['Français', 'English']",7.3,1768.0,70.0,69.0
1,\n\nFin of red 1957 Chevy Impala convertible d...,12,[comedy],125120.0,2009.0,0.0,"['Drama', 'Crime']",12,"['venezia 64', 'Mosfilm']",2007-06-06,7537453.0,159.0,"['', 'Pусский', 'Český']",6.6,54.0,0.0,0.0
2,\n\n ...,12-and-holding,[drama],96464.0,2006.0,0.0,['Drama'],Twelve and Holding,['Serenade Films'],2005-09-11,95687.0,90.0,['English'],7.0,20.0,74.0,80.0
3,\n\n\t\t An original screenplay by\n\...,12-monkeys,"[drama, thriller, science-fiction]",57141459.0,1995.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,88.0
4,\n \n\n\n\n\n ...,127-hours,"[drama, adventure, thriller]",18335230.0,2010.0,0.0,"['Drama', 'Adventure', 'Thriller']",127 Hours,"['Fox Searchlight Pictures', 'Cloud Eight Film...",2010-11-05,35692920.0,94.0,['English'],7.0,2668.0,93.0,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,\n\n Wr...,yes-man,"[comedy, romance]",97690976.0,2008.0,0.0,['Comedy'],Yes Man,"['Village Roadshow Pictures', 'The Zanuck Comp...",2008-12-09,225990978.0,104.0,"['한국어/조선말', 'English', 'Eesti']",6.4,1869.0,46.0,66.0
953,"""YOU CAN COU...",you-can-count-on-me,[drama],9183362.0,2000.0,0.0,"['Drama', 'Romance']",You Can Count on Me,"['Cappa Productions', 'Hart-Sharp Entertainmen...",2000-11-17,0.0,111.0,['English'],6.8,89.0,95.0,88.0
954,\n\n\n\n\n\n ...,youth-in-revolt,"[romance, drama, comedy]",15281286.0,2010.0,0.0,"['Drama', 'Comedy', 'Romance']",Youth in Revolt,['Dimension Films'],2009-09-11,0.0,87.0,"['Français', 'English']",5.9,323.0,66.0,52.0
955,\t\t\tYou've Got Mail\n\n\t\t\tby Nora Ephron ...,youve-got-mail,"[romance, comedy]",115821495.0,1998.0,0.0,"['Comedy', 'Romance']",You've Got Mail,['Warner Bros.'],1998-12-17,250821495.0,119.0,['English'],6.3,859.0,69.0,73.0


In [193]:
merged_df.to_csv('merged_box_mv_rt_scripts.csv')